In [ ]:
import numpy as np
import tensorflow as tf
import sklearn as sk
import os
import unidecode
import pandas as pd
import nltk
import re
from os import listdir
from os.path import isfile, join

In [ ]:
#simple definitions for file io
def read_file(file_name):
    file = unidecode.unidecode(open(file_name).read())
    return file

def read_csv(file_name, sep=','):
    df = pd.read_csv(file_name, sep=sep)
    return df


https://www.kaggle.com/albertsuarez/azlyrics
<h3>
From this webpage, we download all 27 files to be processed and eventually used as dataset for classifier training.
Some simple excel formatting is applied to achieve consistent dataframe shape when reading in the data
</h3>

<h3> The following block of code extracts, from the OHHLa.txt file, all rap songs titles and their respective artists and writes this information into artists_songs.txt file </h3>

In [ ]:
data = pd.read_csv('Resources/All lyrics OHHLa.txt', sep='\n\n\n', engine='python',encoding='utf8', header = 0)
songlist=[]
artistlist = []
print(data)
linelist= data.iloc[:, 0]
filter_a=["Artist:"]
filter_s=["Song:"]

#print(linelist[3])
for k in range(len(linelist)):
    if any(i in linelist[k] for i in filter_a):
        a = linelist[k]
        a = a.lstrip()
        a = a.lower()
        ind1 = a.find('artist:')
        if ind1 != -1:
            a = a[ind1+7:]
        ind2 = a.find('f/')
        if ind2 != -1:
            a = a[0:ind2]
        ind3 = a.find('(')
        if ind3 != -1:
            a = a[0:ind3]
        a = a.replace(" ", "")
        a = a.replace("'", "")
        artistlist.append(a.lstrip().lower())
        
        for kk in range(k+1,k+4):
            if any(j in linelist[kk] for j in filter_s):
                s = linelist[kk]
                s = s.lstrip()
                s = s.lower()
                ind4 = s.find('song:')
                if ind4 != -1:
                    s = s[ind4+5:]
                ind5 = s.find('(')
                if ind5 != -1:
                    s = a[0:ind5]
                s = s.replace(" ", "")
                s = s.replace("'","")
 
                songlist.append([a,s])

artistlist = sorted(set(artistlist))

with open('Resources/artists_songs.txt', 'w') as f:
    f.writelines("artist/song\n")
    for item in songlist:
        f.writelines("%s\n" % item)
        
with open('Resources/artists_names.txt', 'w') as f:
    f.writelines("artist\n")
    for item in artistlist:
        f.writelines("%s\n" % item)

<h3>Given the OHHLA file containing 'all' rap songs, the following blocks are executed to filter out the rap songs in the azlyrics dataset.</h3>

In [ ]:
#from the text file containing all the raps songs and their respective artists (from OHHLA),
#read in the content
raw = read_csv("Resources/artists_songs.txt",'\n')

curr_arr = []
artists_songs = []
raprows = []

for item in raw['artist/song'].values:
    item = item.replace("'","")
    artists_songs.append(item.strip("][").split(', '))

In [ ]:
#this snippet exclusively deals with the azlyrics file for artists whose alias/name starts with 0-9

"""following snippet for initials 0-9"""

file_root1 = 'Resources/azlyrics_lyrics_'
file_format = '.csv'
for item in artists_songs:
    if item[0] == '':
        continue
    for ini in ['0','1','2','3','4','5','6','7','8','9']:
        if item[0][0] == ini:
            curr_arr.append(item)

file_name = file_root1 + '19' + file_format
file = read_csv(file_name)

#if both artist's name and song matches, then add it to the rap list
for item in curr_arr:
    for index, row in file.iterrows():
        if row['ARTIST_NAME'].lower().replace(" ","") == item[0] and row['SONG_NAME'].lower().replace(" ","") == item[1]:
            raprows.append(row.values)
            break


rap_df = pd.DataFrame(raprows, columns=file.columns)   
rap_df.to_csv (r'Resources/rap.csv', index = True, header=True)
"""0-9 ends here"""

In [ ]:
#this snippet deals with the azlyrics files for artists whose alias/name starts with an alphabet

"""all other alphabets start here"""
initials = ['a','b','c','d','e','f','g','h','i','j','k','l','m',\
                  'n','o','p','q','r','s','t','u','v','w','x','y','z']
for ini in initials:
    print("currently processing: ", ini)
    raprows = []
    nraprows = []
    curr_arr = []
    
    for item1 in artists_songs:
        if item1[0] == '':
            continue
            
        if item1[0][0] == ini:
            curr_arr.append(item1)

    file_name = file_root1 + ini + file_format
    file = read_csv(file_name)

    for item2 in curr_arr:
        for index, row in file.iterrows():
            if row['ARTIST_NAME'].lower().replace(" ","") == item2[0] and row['SONG_NAME'].lower().replace(" ","") == item2[1]:
                raprows.append(row.values)
                print(item2[1])
                break
                
    rap_df = pd.DataFrame(raprows, columns=file.columns)   
    rap_df.to_csv (r'Resources/rap.csv', index = True, mode='a', header=False)
"""other alphabets end here"""

<h3>The following blocks of code extract the non-rap songs and compile them into separate csv files.
The criterion for non-rap songs is that the artist's name must not appear at all in the
'artist_names.txt' file. Takes more than a few hours to execute completely.</h3>

In [ ]:
#this snippet exclusively deals with the azlyrics file for artists whose alias/name starts with 0-9

names = []
raw = read_csv("Resources/artists_names.txt",'\n')
for item in raw['artist'].values:
    item = item.lstrip().replace("'","")
    item = item.strip("][").split(', ')[0][:item.find(',')]
    names.append(item[:item.find('/')])
    
print(names)
    

file_root1 = 'Resources/azlyrics_lyrics_'
file_format = '.csv'

curr_names = []
to_drop = []
for item in names:
    for ini in ['0','1','2','3','4','5','6','7','8','9']:
        if (len(item)!= 0) and (item[0] == ini):
            curr_names.append(item)

file_name = file_root1 + '19' + file_format
entries = read_csv(file_name)

for item in curr_names:
    for index, row in entries.iterrows():
        if row['ARTIST_NAME'].lower().replace(" ","").replace(",","") == item: 
            to_drop.append(index)
            print(index, "dropped")
             
nrapdf = entries.drop(to_drop) 
nrapdf.to_csv (r'Resources/nrap.csv', index = False, header=True)


In [ ]:
#this snippet deals with the azlyrics files for artists whose alias/name starts with an alphabet

for ini in initials:
    curr_names = []
    to_drop = []
    print("currently processing: ", ini)
    
    for item in names:
        if (len(item)!= 0) and (item[0] == ini):
            curr_names.append(item)

    file_name = file_root1 + ini + file_format
    entries = read_csv(file_name)

    for item in curr_names:
        for index, row in entries.iterrows():
            if row['ARTIST_NAME'].lower().replace(" ","").replace(",","") == item: 
                to_drop.append(index)

    nrapdf = entries.drop(to_drop) 
    nrapdf.to_csv (r'Dataset/AZlyrics/filtered/nrap.csv', index = False, mode='a',header=False)

<h3> The csv file containing only rap song information is named rap.csv, the compiled list of non-rap song information is split into 3 separate csv files to reduce lag and the chances of crashing: nrap.csv, nrap2.csv,
nrap3.csv. Some manual inspection is done to delete big chunks of foreign language songs, some will remain. </h3>